Enrichment
==========

Dependencies
------------

In [1]:
%AddDeps org.bdgenomics.adam adam-core-spark2_2.11 0.24.0
%AddDeps comp.bio.aging adam-playground_2.11 0.0.13 --repository https://dl.bintray.com/comp-bio-aging/main/

Marking org.bdgenomics.adam:adam-core-spark2_2.11:0.24.0 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir8172174803993100516/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree-tmp-dir8172174803993100516/toree_add_deps/https/repo1.maven.org/maven2/org/bdgenomics/adam/adam-core-spark2_2.11/0.24.0/adam-core-spark2_2.11-0.24.0.jar
-> New file at /tmp/toree-tmp-dir8172174803993100516/toree_add_deps/https/repo1.maven.org/maven2/org/bdgenomics/adam/adam-core-spark2_2.11/0.24.0/adam-core-spark2_2.11-0.24.0.pom


Waiting for a Spark session to start...

Marking comp.bio.aging:adam-playground_2.11:0.0.13 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir8172174803993100516/toree_add_deps/
-> https://dl.bintray.com/comp-bio-aging/main/
-> https://repo1.maven.org/maven2


In [2]:
%AddDeps org.apache.hadoop hadoop-azure 2.7.6
%AddDeps com.microsoft.azure azure-storage 2.0.0

Marking org.apache.hadoop:hadoop-azure:2.7.6 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir8172174803993100516/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree-tmp-dir8172174803993100516/toree_add_deps/https/repo1.maven.org/maven2/org/apache/hadoop/hadoop-azure/2.7.6/hadoop-azure-2.7.6.jar
-> New file at /tmp/toree-tmp-dir8172174803993100516/toree_add_deps/https/repo1.maven.org/maven2/org/apache/hadoop/hadoop-azure/2.7.6/hadoop-azure-2.7.6.pom


Waiting for a Spark session to start...

Marking com.microsoft.azure:azure-storage:2.0.0 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir8172174803993100516/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree-tmp-dir8172174803993100516/toree_add_deps/https/repo1.maven.org/maven2/com/microsoft/azure/azure-storage/2.0.0/azure-storage-2.0.0.jar
-> New file at /tmp/toree-tmp-dir8172174803993100516/toree_add_deps/https/repo1.maven.org/maven2/com/microsoft/azure/azure-storage/2.0.0/azure-storage-2.0.0.pom


General functions
-----------------

In [3]:
import  org.apache.spark._
import org.apache.spark.sql.{DataFrame, Encoders, SparkSession}
import org.apache.spark.sql.types.StructType
import scala.reflect.runtime.universe._
import org.apache.spark.storage.StorageLevel
import org.apache.spark.rdd._
import org.apache.spark.sql.functions._

In [4]:
def sparkHadoopConf(sc: SparkContext, acountName: String, accountKey: String) = {
  sc.hadoopConfiguration.set("fs.azure", "org.apache.hadoop.fs.azure.NativeAzureFileSystem")
  sc.hadoopConfiguration.set("fs.azure.account.key." + acountName + ".blob.core.windows.net", accountKey)
  sc
}

sparkHadoopConf: (sc: org.apache.spark.SparkContext, acountName: String, accountKey: String)org.apache.spark.SparkContext


In [5]:
def azurize(container: String, accountName: String, blobFile: String): String = "wasbs://"+container+"@"+accountName+".blob.core.windows.net/"+blobFile 

def writeText2Azure[T]( rdd: RDD[T], container: String, accountName: String, blobFile: String ): String =
{
  val url = azurize(container, accountName, blobFile)
  rdd.saveAsTextFile(url)
  url
}

def writeTsv2Azure( df: DataFrame, container: String, accountName: String, blobFile: String ): String =
{
  val url = azurize(container, accountName, blobFile)
  df.write.option("sep","\t").option("header","true").csv(url)
  url
}

azurize: (container: String, accountName: String, blobFile: String)String
writeText2Azure: [T](rdd: org.apache.spark.rdd.RDD[T], container: String, accountName: String, blobFile: String)String
writeTsv2Azure: (df: org.apache.spark.sql.DataFrame, container: String, accountName: String, blobFile: String)String


In [6]:
val account = "pipelines1"
val key = ""
val connString = s"DefaultEndpointsProtocol=https;AccountName=pipelines1;AccountKey=${key};EndpointSuffix=core.windows.net"
def az(path: String): String = azurize("storage", account, path)

account = pipelines1
key = 
connString = DefaultEndpointsProtocol=https;AccountName=pipelines1;AccountKey=;EndpointSuffix=core.windows.net


az: (path: String)String


DefaultEndpointsProtocol=https;AccountName=pipelines1;AccountKey=;EndpointSuffix=core.windows.net

In [7]:
def write(df: DataFrame, url: String, header: Boolean = true) = {
  df.coalesce(1).write.option("sep","\t").option("header", header).csv(url)
  url
}

write: (df: org.apache.spark.sql.DataFrame, url: String, header: Boolean)String


In [8]:
def readTSV(path: String, header: Boolean = false, sep: String = "\t"): DataFrame = spark.read
    .option("sep", sep)
    .option("comment", "#")
    .option("inferSchema", true)
    .option("header", header)
    .option("ignoreLeadingWhiteSpace", true)
    .option("ignoreTrailingWhiteSpace", true)
    .option("ignoreTrailingWhiteSpace", true)
    .option("maxColumns", 150000)
    .csv(path)

readTSV: (path: String, header: Boolean, sep: String)org.apache.spark.sql.DataFrame


In [9]:
def toVectors(dataFrame: DataFrame, columns: Seq[String], output: String) = {  
  import org.apache.spark.ml.feature.VectorAssembler
  import org.apache.spark.ml.linalg.Vectors
  val assembler = new VectorAssembler()
    .setInputCols(columns.toArray)
    .setOutputCol(output)
  assembler.transform(dataFrame.na.fill(0.0, columns).na.fill(0.0)).select(output)
}

toVectors: (dataFrame: org.apache.spark.sql.DataFrame, columns: Seq[String], output: String)org.apache.spark.sql.DataFrame


In [11]:
val sparkContext = sc
sparkHadoopConf(sparkContext, account, key)
  
val spark = SparkSession
  .builder()
  .appName("mapping_models")
  .getOrCreate()
import spark.implicits._

sparkContext = org.apache.spark.SparkContext@6ca639b5
spark = org.apache.spark.sql.SparkSession@3508f046


Spark app-20180706085823-0011: Some(http://c8fc1f462c1d:4040)

In [14]:
def rename(dataFrame: DataFrame, renamings: Map[String, String]) =   {
      val newColumns = dataFrame.columns.map(c=> renamings.getOrElse(c, c))
      dataFrame.toDF(newColumns:_*)
    }
def renameFunction(renamings: Map[String, String]) = udf[String, String](c=> renamings.getOrElse(c, c))

rename: (dataFrame: org.apache.spark.sql.DataFrame, renamings: Map[String,String])org.apache.spark.sql.DataFrame
renameFunction: (renamings: Map[String,String])org.apache.spark.sql.expressions.UserDefinedFunction


In [15]:
val expressionsPath = "expressions"
val byGoPath = expressionsPath + "/go"
val comparison = byGoPath + "/gray_whale_with_bowhead_with_minke_with_NMR_with_human_with_mouse_with_cow_full_outer_counts_extended.tsv"
val grouped = byGoPath + "/grouped"
val ranked = byGoPath + "/grouped/ranked"
val transcriptsPath = expressionsPath + "/transcripts"
val indexesPath = "indexes"
val reactomePath = indexesPath + "/reactome"
val unirefPath = expressionsPath + "/" + "uniref90"

expressionsPath = expressions
byGoPath = expressions/go
comparison = expressions/go/gray_whale_with_bowhead_with_minke_with_NMR_with_human_with_mouse_with_cow_full_outer_counts_extended.tsv
grouped = expressions/go/grouped
ranked = expressions/go/grouped/ranked
transcriptsPath = expressions/transcripts
indexesPath = indexes
reactomePath = indexes/reactome
unirefPath = expressions/uniref90


expressions/uniref90

GO annotations
--------------

In [16]:
%AddDeps net.sansa-stack sansa-rdf-spark_2.11 0.4.0 --transitive
%AddDeps net.sansa-stack sansa-owl-spark_2.11 0.4.0 --transitive
%AddDeps net.sansa-stack sansa-query-spark_2.11 0.4.0

Marking net.sansa-stack:sansa-rdf-spark_2.11:0.4.0 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir8172174803993100516/toree_add_deps/
-> https://repo1.maven.org/maven2
-> Failed to resolve com.ibm.sparktc.sparkbench:sparkbench:2.3.0_0.4.0
    -> not found: /tmp/toree-tmp-dir8172174803993100516/toree_add_deps/cache/com.ibm.sparktc.sparkbench/sparkbench/ivy-2.3.0_0.4.0.xml
    -> not found: https://repo1.maven.org/maven2/com/ibm/sparktc/sparkbench/sparkbench/2.3.0_0.4.0/sparkbench-2.3.0_0.4.0.pom
-> New file at /tmp/toree-tmp-dir8172174803993100516/toree_add_deps/https/repo1.maven.org/maven2/log4j/log4j/1.2.17/log4j-1.2.17.zip
-> New file at /tmp/toree-tmp-dir8172174803993100516/toree_add_deps/https/repo1.maven.org/maven2/org/spark-project/hive/hive-exec/1.2.1.spark2/hive-exec-1.2.1.spark2.jar
-> New file at /tmp/toree-tmp-dir8172174803993100516/toree_add_deps/https/repo1.maven.org/maven2/org/apache/hadoop/hadoop-yarn-common/2.8.3/hadoop-yarn-common-2.8.3.jar
-> New file

error: error while loading StaticSQLConf, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-catalyst_2.11-2.3.0.jar(org/apache/spark/sql/internal/StaticSQLConf.class)' has location not matching its contents: contains class StaticSQLConf
error: error while loading SessionStateBuilder, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-sql_2.11-2.3.0.jar(org/apache/spark/sql/internal/SessionStateBuilder.class)' has location not matching its contents: contains class SessionStateBuilder
error: error while loading HiveSerDe, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-sql_2.11-2.3.0.jar(org/apache/spark/sql/internal/HiveSerDe.class)' has location not matching its contents: contains class HiveSerDe
error: error while loading CatalogImpl, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-sql_2.11-2.3.0.jar(org/apache/spark/sql/internal/CatalogImpl.class)' has location not matching its contents: contains class CatalogImpl
error: error while loading 

Marking net.sansa-stack:sansa-owl-spark_2.11:0.4.0 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir8172174803993100516/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree-tmp-dir8172174803993100516/toree_add_deps/https/repo1.maven.org/maven2/org/apache/spark/spark-core_2.11/2.3.1/spark-core_2.11-2.3.1.jar
-> New file at /tmp/toree-tmp-dir8172174803993100516/toree_add_deps/https/repo1.maven.org/maven2/org/apache/commons/commons-math3/3.4.1/commons-math3-3.4.1.jar
-> New file at /tmp/toree-tmp-dir8172174803993100516/toree_add_deps/https/repo1.maven.org/maven2/org/apache/hadoop/hadoop-mapreduce-client-core/2.7.0/hadoop-mapreduce-client-core-2.7.0.jar
-> New file at /tmp/toree-tmp-dir8172174803993100516/toree_add_deps/https/repo1.maven.org/maven2/org/apache/hadoop/hadoop-common/2.7.0/hadoop-common-2.7.0.jar
-> New file at /tmp/toree-tmp-dir8172174803993100516/toree_add_deps/https/repo1.maven.org/maven2/org/slf4j/jul-to-slf4j/1.7.16/jul-to-slf4j-1.7.1

In [17]:
import net.sansa_stack.rdf.spark.io._
import org.apache.jena.riot.Lang

val input = az("/go/go.owl")

val lang = Lang.RDFXML
val triples = spark.rdf(lang)(input)

input = wasbs://storage@pipelines1.blob.core.windows.net//go/go.owl
lang = Lang:RDF/XML
triples = MapPartitionsRDD[1] at map at package.scala:225


MapPartitionsRDD[1] at map at package.scala:225

In [23]:
import net.sansa_stack.rdf.spark.model._
import net.sansa_stack.query.spark.query

import org.apache.jena.graph.NodeFactory
def toGO(id: String) = NodeFactory.createURI(id.replace("GO:","http://purl.obolibrary.org/obo/GO_"))

toGO: (id: String)org.apache.jena.graph.Node


In [19]:
import net.sansa_stack.rdf.spark.model.graph._
val g = GraphOps.constructStringGraph(triples).cache
g

g = org.apache.spark.graphx.impl.GraphImpl@9009ebf


org.apache.spark.graphx.impl.GraphImpl@9009ebf

In [20]:
import org.apache.jena.graph.NodeFactory
def uri(str: String) = NodeFactory.createURI(str)
def toGO(id: String) = uri(id.replace("GO:","http://purl.obolibrary.org/obo/go#"))
val alternative = uri("http://www.geneontology.org/formats/oboInOwl#hasAlternativeId")
val namespace =  uri("http://www.geneontology.org/formats/oboInOwl#hasOBONamespace")


alternative = http://www.geneontology.org/formats/oboInOwl#hasAlternativeId
namespace = http://www.geneontology.org/formats/oboInOwl#hasOBONamespace


error: error while loading EdgeDirection, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-graphx_2.11-2.3.0.jar(org/apache/spark/graphx/EdgeDirection.class)' has location not matching its contents: contains class EdgeDirection
error: error while loading EdgeTriplet, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-graphx_2.11-2.3.0.jar(org/apache/spark/graphx/EdgeTriplet.class)' has location not matching its contents: contains class EdgeTriplet
error: error while loading GraphOps, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-graphx_2.11-2.3.0.jar(org/apache/spark/graphx/GraphOps.class)' has location not matching its contents: contains class GraphOps
error: error while loading PartitionStrategy, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-graphx_2.11-2.3.0.jar(org/apache/spark/graphx/PartitionStrategy.class)' has location not matching its contents: contains trait PartitionStrategy
error: error while loading TripletFields, class file 

http://www.geneontology.org/formats/oboInOwl#hasOBONamespace

In [21]:
val namespaces = triples.toDF.where($"p" === "http://www.geneontology.org/formats/oboInOwl#hasOBONamespace").select("s", "o").withColumnRenamed("o", "namespace")
val labels = triples.toDF.where($"p" === "http://www.w3.org/2000/01/rdf-schema#label").select("s", "o").withColumnRenamed("o", "label")
def toGO = udf[String, String](str => str.replace("http://purl.obolibrary.org/obo/GO_", "GO:"))

val go_anno = namespaces.join(labels, Seq("s")).withColumn("GO", toGO($"s")).select("GO", "namespace", "label").cache

namespaces = [s: string, namespace: string]
labels = [s: string, label: string]
go_anno = [GO: string, namespace: string ... 1 more field]


error: error while loading QualifiedTableName, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-catalyst_2.11-2.3.0.jar(org/apache/spark/sql/catalyst/QualifiedTableName.class)' has location not matching its contents: contains class QualifiedTableName
error: error while loading JavaTypeInference, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-catalyst_2.11-2.3.0.jar(org/apache/spark/sql/catalyst/JavaTypeInference.class)' has location not matching its contents: contains class JavaTypeInference
error: error while loading FunctionIdentifier, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-catalyst_2.11-2.3.0.jar(org/apache/spark/sql/catalyst/FunctionIdentifier.class)' has location not matching its contents: contains class FunctionIdentifier
error: error while loading DefinedByConstructorParams, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-catalyst_2.11-2.3.0.jar(org/apache/spark/sql/catalyst/DefinedByConstructorParams.class)' has location 

[GO: string, namespace: string ... 1 more field]

In [ ]:
%%dataframe go_anno

In [ ]:
write(go_anno, az("/go/names.tsv"), true)

In [ ]:
/*
def toGO(str: String) = str.replace("http://purl.obolibrary.org/obo/GO_", "GO:")
val ns = nms.map{ case (key, value) => toGO(key) -> value}.toDF("go", "namespace")
val newColumns = "namespace"::comparison.columns.toList.tail.map(s=> if(s=="gray_whale_average") "gray_whale_avg" else s)
val table = comparison.join(ns, Seq("go"), "left_outer")
  .withColumn("gray_whale_avg", toDouble($"gray_whale_average")).drop("gray_whale_average")
  .select("go", newColumns:_*)
  .sort($"gray_whale_avg".desc)
table
*/

GO enrichment
=============

In [ ]:
val goTable = readTSV(az(comparison), true)

In [ ]:
%%dataframe goTable

In [ ]:
val components = readTSV(az(grouped + "/by_biological_process.tab"  ), true)
val processes = readTSV(az(grouped + "/by_cellular_component.tab"  ), true)
val functions = readTSV(az(grouped + "/by_molecular_function.tsv"  ), true)

In [ ]:
(components.count, processes.count, functions.count, goTable.count)

In [ ]:
import org.apache.spark.sql._
import org.apache.spark.sql.catalyst.encoders.RowEncoder

def goFlatten(df: DataFrame) = {
    df.select("GO").na.fill("").flatMap{
      case Row(g: String) =>
        val goes = if(g==null) Array[(String, Int)]() else g.split(";").map(go=>go.trim -> 1).filter{ case (go, _) => go!=""}
        goes
    }.join(go_anno, Seq("go"))
}
def goCounts(df: DataFrame) = {
    goFlatten(df)
        .groupByKey(_._1)
        .mapValues(_._2)
        .reduceGroups(_ + _)
        .toDF("GO", "GO_count")
        .sort($"GO_count".desc)
}

val process = "biological_process"
val function = "molecular_function"
val component = "cellular_component"

def goFilteredCounts(df: DataFrame, namespace: String)  = {
    val flat = goFlatten(df).join(ns, Seq("GO"))
    flat
        .where($"namespace" = namespace)
        .groupByKey(_._1)
        .mapValues(_._2)
        .reduceGroups(_ + _)
        .toDF("GO", "GO_count")
        .sort($"GO_count".desc)
}

In [ ]:
import org.apache.commons.math3.distribution.{HypergeometricDistribution}

def enrichmentFunction = udf[Double, Int, Int, Int, Int]{
    case (populationSize: Int, numberOfSuccesses: Int, sampleSize: Int, value: Int) => 
      val a = new HypergeometricDistribution(populationSize, numberOfSuccesses, sampleSize)
      1 - a.cumulativeProbability(value-1)
  }
def significantFunction = udf[Boolean, Double]{ v => v <= 0.01}

In [ ]:
def topExpressed(df: DataFrame, name: String, n: Int) = {
  val sorted = df.sort(new ColumnName(name).desc)
  sorted.select("GO").limit(n)
}

def prepareEnrichment(df: DataFrame, name: String, n: Int) = {
    val allCounts: DataFrame = goCounts(df)
    import org.apache.spark.sql.functions.sum
    val population = allCounts.agg(sum("GO_count")).as[Long].head
    val countName = s"top_${name}_${n}_GO_count"
    val tc = goCounts(topExpressed(df, name, n))
      .withColumnRenamed("GO_count", countName)
    allCounts
        .withColumn("all_GO", lit(population))
        .withColumn("sample_size", lit(n))
        .join(tc, Seq("GO"))
    .sort(new ColumnName(countName).desc)
}

def enrich(df: DataFrame, name: String, n: Int) = {
    val data = prepareEnrichment(df, name, n)
    val countName = s"top_${name}_${n}_GO_count"
    data
    .withColumn("p-value", enrichmentFunction($"all_GO", $"GO_count", $"sample_size",  new ColumnName(countName)))
    .withColumn("significant", significantFunction($"p-value"))
    .sort($"p-value".desc)
}

In [ ]:
def prepareEnrichmentNamespaced(df: DataFrame, name: String, n: Int) = {
    val allCounts: DataFrame = goCounts(df)
    import org.apache.spark.sql.functions.sum
    val population = allCounts.agg(sum("GO_count")).as[Long].head
    val countName = s"top_${name}_${n}_GO_count"
    val tc = goCounts(topExpressed(df, name, n))
      .withColumnRenamed("GO_count", countName)
    allCounts
        .withColumn("all_GO", lit(population))
        .withColumn("sample_size", lit(n))
        .join(tc, Seq("GO"))
    .sort(new ColumnName(countName).desc)
}

def enrich(df: DataFrame, name: String, namespace: String, n: Int) = {
    val data = prepareEnrichment(df, name, n)
    val countName = s"top_${name}_${n}_GO_count"
    data
    .withColumn("p-value", enrichmentFunction($"all_GO", $"GO_count", $"sample_size",  new ColumnName(countName)))
    .withColumn("significant", significantFunction($"p-value"))
    .sort($"p-value".desc)
}

In [ ]:
val g = enrich(gray_whale, "liver", 50)

In [ ]:
%%dataframe g

In [ ]:
(g.count, g.where("significant = true").count, g.where("significant = false").count)

In [ ]:
val g = enrich(gray_whale, "kidney", 200)

In [ ]:
(g.count, g.where("significant = true").count, g.where("significant = false").count)

In [ ]:
val allUniref = readTSV(az(unirefPath + "/all_uniref90_outer.tab"  ), true)

In [ ]:
val gray_whale = readTSV(az(unirefPath + "/gray_whale_uniref90_all.tab"  ), true)

In [ ]:
%%dataframe gray_whale_counts

In [ ]:
val gray_whale_simple = gray_whale.select("uniref90", "go", "liver", "kidney").na("")

In [ ]:
%%dataframe gray_whale_simple

In [ ]:
%%dataframe allUniref

In [3]:
%%dataframe gray_whale_simple.select("uniref90")

An error occurred converting DataFrame to html.
<console>:26: error: not found: value gray_whale_simple
       gray_whale_simple.select("uniref90")
       ^


In [32]:
import org.apache.commons.math3.distribution.{HypergeometricDistribution}

def makeDistrib(populationSize: Int, numberOfSuccesses: Int, sampleSize: Int): HypergeometricDistribution ={
  val a = new ApacheHypergeometricDistribution(populationSize, numberOfSuccesses, sampleSize)
  a
}

Name: Compile Error
Message: <console>:92: error: not found: type ApacheHypergeometricDistribution
         val a = new ApacheHypergeometricDistribution(populationSize, numberOfSuccesses, sampleSize)
                     ^
<console>:92: error: <none> is abstract; cannot be instantiated
         val a = new ApacheHypergeometricDistribution(populationSize, numberOfSuccesses, sampleSize)
                 ^

StackTrace: 

In [24]:
def geomPValue(h: HypergeometricDistribution) = (0 to 100).foldLeft(0.0){ case (acc, el) => acc + h.probabilityOf(el)}

Name: Compile Error
Message: <console>:89: error: not found: type HypergeometricDistribution
       def geomPValue(h: HypergeometricDistribution) = (0 to 100).foldLeft(0.0){ case (acc, el) => acc + h.probabilityOf(el)}
                         ^

StackTrace: 

In [25]:
def enrichment(population: Int, sampleSize: Int = 100) = udf[Double, Double](c=> renamings.getOrElse(c, c))

Name: Compile Error
Message: <console>:89: error: not found: value renamings
       def enrichment(population: Int, sampleSize: Int = 100) = udf[Double, Double](c=> renamings.getOrElse(c, c))
                                                                                        ^

StackTrace: 

In [1]:
println("Helo")

Helo


In [2]:
 allUniref.count

Name: Compile Error
Message: <console>:26: error: not found: value allUniref
       allUniref.count
       ^

StackTrace: 